In [ ]:
import xml.etree.ElementTree as elemTree
import base64, struct, binascii
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import glob, os 
import numpy as np
import pandas as pd
pd.set_option('display.max.colwidth', 500)

In [ ]:
# 원본 xml 데이터 위치
xml_2013 = glob.glob('/data/hw/2013/*.xml')

In [ ]:
xml_2013 = os.listdir('/data/2015/**/*.xml', recursive=True)


In [ ]:
import xml.etree.ElementTree as elemTree
import pandas as pd
import glob
import base64
import numpy as np

# 모든 하위 폴더에서 XML 파일 검색
xml_pattern = '/data/hw/2022/**/*.xml'
xml_files = glob.glob(xml_pattern, recursive=True)

# CSV 파일 경로
csv_file_path = '/home/ubuntu/hw/12Lead+Mobile_ECG/orig_20230816/label/Total_Mobile_label.csv'

# CSV 파일 로드
df_csv = pd.read_csv(csv_file_path)

# NSR이 1이고 AF가 0인 PID 필터링
filtered_pids = df_csv[(df_csv['NSR'] == 1) & (df_csv['AF'] == 0)]['PID'].astype(str).tolist()

# 검색된 파일 경로 출력
for file in xml_files:
    print(file)

# 필요한 데이터를 저장하기 위한 빈 리스트들을 초기화합니다.
pid, date, diag, xml_paths, gender, age, name = [], [], [], [], [], [], []
vent_rate, atrial_rate, QRS_duration, QT_int, QT_corr, PR_int, P_axis, R_axis, T_axis, QRS_count = [], [], [], [], [], [], [], [], [], []
Q_on, Q_off, P_on, P_off, T_off, QT_cFrederica = [], [], [], [], [], []
lead1, lead2, leadv1, leadv2, leadv3, leadv4, leadv5, leadv6 = [], [], [], [], [], [], [], []
leadIII, leadaVR, leadaVL, leadaVF = [], [], [], []

# XML 파일 리스트를 순회하면서 각 파일을 처리합니다.
for i, xml_file in enumerate(xml_files):
    # 진행 상황을 출력합니다. 매 50000번째 파일마다 로그를 남깁니다.
    if (i+1) % 50000 == 0:
        print(f"Processing file {i+1}")

    # XML 파일을 파싱합니다. root 변수에 XML의 루트 요소를 저장합니다.
    tree = elemTree.parse(xml_file)
    root = tree.getroot()
    
    # 환자 ID를 찾습니다. 'NO PID'인 경우 또는 유효하지 않은 경우 건너뜁니다.
    PID = root.find('PatientDemographics').findtext('PatientID')
    if PID == 'NO PID' or PID not in filtered_pids:
        continue
        
    try:
        # PID가 정수인지 확인합니다. 정수가 아닌 경우 예외를 발생시킵니다.
        int(PID)
        
        # 파형 데이터를 포함하는 XML 섹션을 찾습니다.
        waveform = root.findall('Waveform')
        # PID 길이가 8이고, 파형 데이터가 8개의 리드로 구성되어 있는지 확인합니다.
        if (len(PID) == 8) & (len(waveform[1].findall('LeadData')) == 8):
            # 날짜와 진단명을 처리합니다.
            stmt = root.find('Diagnosis').findall('DiagnosisStatement')
            statement = ', '.join([d.find('StmtText').text for d in stmt])
            diag.append(statement)
            
            # 환자 인구 통계학적 정보를 추출합니다.
            name.append(root.find('PatientDemographics').findtext('PatientLastName'))
            pid.append(PID)
            age.append(root.find('PatientDemographics').findtext('PatientAge'))
            
            # 성별 정보를 추출합니다.
            gender_text = root.find('PatientDemographics').findtext('Gender')
            gender.append(gender_text)
            
            # ECG 측정값들을 추출합니다.
            vent_rate.append(root.find('RestingECGMeasurements').findtext('VentricularRate'))

            # 취득 날짜와 시간을 포맷팅합니다.
            Date = root.find('TestDemographics').findtext('AcquisitionDate')
            Time = root.find('TestDemographics').findtext('AcquisitionTime')
            date.append(Date + " " + Time)

            # XML 파일 경로를 저장합니다.
            xml_paths.append(xml_file)
            
            # 각 리드에 대한 파형 데이터를 추출하고 Base64로 디코딩합니다.
            leads = {}  # 각 파일 처리 시작 시 leads 딕셔너리 초기화
            for lead in waveform[1].findall('LeadData'):
                lead_id = lead.findtext('LeadID')
                waveform_data_encoded = lead.findtext('WaveFormData')
                
                # Base64로 인코딩된 파형 데이터 디코딩
                waveform_data_decoded = base64.b64decode(waveform_data_encoded)
                waveform_data_array = np.frombuffer(waveform_data_decoded, dtype=np.int16)  # numpy 배열로 변환
        
                # 처리된 데이터를 딕셔너리에 저장
                leads[lead_id] = waveform_data_array
            
                # 처리된 데이터를 적절한 리스트에 저장
                if lead_id == 'I':
                    lead1.append(waveform_data_array)
                elif lead_id == 'II':
                    lead2.append(waveform_data_array)
                elif lead_id == 'V1':
                    leadv1.append(waveform_data_array)
                elif lead_id == 'V2':
                    leadv2.append(waveform_data_array)                    
                elif lead_id == 'V3':
                    leadv3.append(waveform_data_array)
                elif lead_id == 'V4':
                    leadv4.append(waveform_data_array)
                elif lead_id == 'V5':
                    leadv5.append(waveform_data_array)
                elif lead_id == 'V6':
                    leadv6.append(waveform_data_array)

            # 새로운 리드 데이터 계산 (모든 리드가 처리된 후에 수행)
            if 'I' in leads and 'II' in leads:
                leads['III'] = np.subtract(leads['II'], leads['I'])
                leads['aVR'] = np.add(leads['I'], leads['II']) * (-0.5)
                leads['aVL'] = np.subtract(leads['I'], 0.5 * leads['II'])
                leads['aVF'] = np.subtract(leads['II'], 0.5 * leads['I'])
            
                # 계산된 리드 데이터를 리스트에 추가
                leadIII.append(leads['III'])
                leadaVR.append(leads['aVR'])
                leadaVL.append(leads['aVL'])
                leadaVF.append(leads['aVF'])
                        
    except ValueError:
        # PID가 정수가 아닐 경우 예외 처리합니다.
        pass
    except AttributeError:
        # XML 구조가 예상과 다를 경우 예외 처리합니다.
        pass

# 데이터를 DataFrame으로 변환
data = {
    'PID': pid,
    'Date': date,
    'Diagnosis': diag,
    'XML': xml_paths,
    'Gender': gender,
    'Age': age,
    'Name': name,
    'LeadI': lead1,
    'LeadII': lead2,
    'LeadIII': leadIII,
    'LeadaVR': leadaVR,
    'LeadaVL': leadaVL,
    'LeadaVF': leadaVF,
    'LeadV1': leadv1,
    'LeadV2': leadv2,
    'LeadV3': leadv3,
    'LeadV4': leadv4,
    'LeadV5': leadv5,
    'LeadV6': leadv6,
}

df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('/home/ubuntu/hw/2022_ecg_patient_data.csv', index=False)

# 기간을 정해서 추출

In [ ]:
# 'Date' 컬럼을 datetime 객체로 변환
df['Date'] = pd.to_datetime(df['Date'])

# 시작 날짜와 종료 날짜를 지정
start_date = '2022-01-01'
end_date = '2022-12-31'

# 지정된 날짜 범위에 해당하는 데이터 필터링
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

# 결과 확인
filtered_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# 필요한 리드 데이터 선택
leads = ['LeadI', 'LeadII', 'LeadIII', 'LeadaVR', 'LeadaVL', 'LeadaVF', 'LeadV1', 'LeadV2', 'LeadV3', 'LeadV4', 'LeadV5', 'LeadV6']

# 시각화 설정
plt.figure(figsize=(20, 15))  # 그래프 크기 설정

# 각 리드에 대해 반복하여 그래프 그리기
for i, lead in enumerate(leads, 1):
    plt.subplot(len(leads), 1, i)  # 여러 그래프를 나란히 배치
    # 각 리드의 데이터를 그래프에 그리기
    lead_data = df[lead].iloc[0]
    plt.plot(lead_data)
    plt.title(lead) 
    

# 각 그래프에 리드 이름을 제목으로 설정

plt.tight_layout()
plt.show()


In [ ]:
from ECGXMLReader import ECGXMLReader

ecg = ECGXMLReader('/home/ubuntu/hw/ecg_sample/MUSE_20210720_132356_07000.xml', augmentLeads=True)


try:
    print(ecg.getLeadVoltages('I'))
except AttributeError:
    print("LeadVoltages 속성이 초기화되지 않았습니다. 데이터를 먼저 로드하거나 파싱해야 할 수 있습니다.")


In [ ]:
print('pid',len(pid))
print('date',len(date))
# print('date'len(date))
print('diag',len(diag))
print('xml',len(xml))
print('age',len(age))
print('gender',len(gender))
print('name',len(name))
print('vent_rate',len(vent_rate))

In [16]:
total_data = pd.read_csv('/home/ubuntu/hw/Total_Mobile_label.csv')

In [ ]:
import pandas as pd

# data2022와 total_data가 두 데이터프레임이라고 가정합니다.
# 두 데이터프레임의 'PID' 열을 문자열 타입으로 변환합니다.
df['PID'] = df['PID'].astype(str)
total_data['PID'] = total_data['PID'].astype(str)

# total_data의 각 'PID'가 data2022에 존재하는지 확인합니다.
common_pids = df[df['PID'].isin(total_data['PID'])]

# 결과 출력
if not common_pids.empty:
    print("data2022에 존재하는 PID:")
    print(common_pids['PID'])
else:
    print("data2022에 존재하는 PID가 없습니다.")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 데이터프레임 df 로드 (df는 이미 데이터를 포함하고 있다고 가정)
# 예: df = pd.read_csv('your_data.csv') 또는 다른 데이터 로딩 방법

# 리드 이름
leads = ['LeadI', 'LeadII', 'LeadIII', 'LeadaVR', 'LeadaVL', 'LeadaVF', 'LeadV1', 'LeadV2', 'LeadV3', 'LeadV4', 'LeadV5', 'LeadV6']

# 각 리드 데이터를 배열로 변환
lead_data_arrays = [df[lead].iloc[0] for lead in leads]

# 각 리드에 대한 데이터를 시각화
plt.figure(figsize=(20, 10))  # 그래프 크기 설정
for i, lead in enumerate(leads):
    plt.subplot(len(leads), 1, i + 1)  # 각 리드에 대한 서브플롯 생성
    plt.plot(lead_data_arrays[i])  # 리드 데이터 플롯
    plt.title(lead)  # 서브플롯 제목 설정
    plt.ylabel('Amplitude')  # y축 레이블 설정

plt.xlabel('Time')  # x축 레이블 설정
plt.tight_layout()  # 레이아웃 조정
plt.show()  # 그래프 표시


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# Assuming 'df' and 'total_data' are your DataFrames
leads = ['LeadI', 'LeadII', 'LeadIII', 'LeadaVR', 'LeadaVL', 'LeadaVF', 'LeadV1', 'LeadV2', 'LeadV3', 'LeadV4', 'LeadV5', 'LeadV6']

# Convert 'PID' columns to string in both DataFrames
df['PID'] = df['PID'].astype(str)
total_data['PID'] = total_data['PID'].astype(str)

# Find common PIDs between df and total_data
common_pids = df[df['PID'].isin(total_data['PID'])]['PID']

# Counter to track the occurrence of each PID
pid_counter = Counter()

# Check if there are common PIDs and process them
if not common_pids.empty:
    print("Processing common PIDs...")
    for pid in common_pids:
        # Update the counter for this PID
        pid_counter[pid] += 1

        # Extract the lead data for the current pid
        lead_data = df[df['PID'] == pid][leads]

        # Convert each lead data to an array (assuming they are stored as lists or similar structures)
        lead_data_arrays = [np.array(lead_data[lead].iloc[0]) for lead in leads]

        # Combine all lead data arrays into a single 2D array
        combined_lead_data = np.array(lead_data_arrays)

        # Generate a unique file name for this PID
        filename = f'/data/12lead_data_2021/{pid}_{pid_counter[pid]}.npy'

        # Save the combined numpy array to the unique file name
        np.save(filename, combined_lead_data)
else:
    print("No common PIDs found between df and total_data.")


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# 'df'와 'total_data'가 여러분의 DataFrame이라고 가정합니다.
leads = ['LeadI', 'LeadII', 'LeadIII', 'LeadaVR', 'LeadaVL', 'LeadaVF', 'LeadV1', 'LeadV2', 'LeadV3', 'LeadV4', 'LeadV5', 'LeadV6']

# 두 DataFrame에서 'PID' 컬럼을 문자열로 변환합니다.
filtered_df['PID'] = filtered_df['PID'].astype(str)
total_data['PID'] = total_data['PID'].astype(str)

# df와 total_data 사이에 공통된 PID 찾기
common_pids = filtered_df[filtered_df['PID'].isin(total_data['PID'])]['PID'].unique()

# 공통된 PID가 있고, 이를 처리합니다.
if len(common_pids) > 0:
    print("공통 PID 처리 중...")
    for pid in common_pids:
        # 현재 pid에 대한 리드 데이터를 추출합니다. 여기서는 첫 번째 일치하는 PID만 처리합니다.
        lead_data = filtered_df[filtered_df['PID'] == pid][leads]

        # 각 리드 데이터를 배열로 변환합니다 (리스트나 유사한 구조로 저장되어 있다고 가정).
        # 첫 번째 일치하는 행만 사용합니다.
        lead_data_arrays = [np.array(lead_data[lead].iloc[0]) for lead in leads]

        # 모든 리드 데이터 배열을 하나의 2D 배열로 결합합니다.
        combined_lead_data = np.array(lead_data_arrays)

        # 이 PID에 대한 파일 이름을 생성합니다. 이미 처리된 PID는 무시합니다.
        filename = f'/data/12lead_data_2021_2022_0217/{pid}.npy'

        # 결합된 numpy 배열을 파일 이름으로 저장합니다.
        # 파일이 이미 존재하지 않는 경우에만 저장합니다.
        if not os.path.exists(filename):
            np.save(filename, combined_lead_data)
            print(f'{filename} 저장됨')
else:
    print("df와 total_data 사이에 공통된 PID가 없습니다.")


In [ ]:
data = np.load('/home/ubuntu/hw/12Lead+Mobile_ECG/orig_20230816/data/Mobile_ECG_ScaledALL/',allow_pickle=True)


In [ ]:
plt.plot(data[2])

In [ ]:
path = '/data/hw/2014/'

In [ ]:
print(len(path))